## Wikipedia module

In [35]:
from langchain.retrievers import WikipediaRetriever
from langchain.tools import Tool, DuckDuckGoSearchRun
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import requests
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
llm = ChatGroq(model="gpt-4o", temperature=0)

In [36]:
YOUTUBE_API_KEY = "your-youtube-api-key"
youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)

In [37]:
class YouTubeSearchTool:
    def __init__(self, max_results=2):
        self.max_results = max_results
        
    def search(self, query):
        try:
            # Search for videos
            search_response = youtube.search().list(
                q=query,
                part='id,snippet',
                maxResults=self.max_results,
                type='video'
            ).execute()
            
            results = []
            for item in search_response.get('items', []):
                video_id = item['id']['videoId']
                title = item['snippet']['title']
                
                # Get transcript
                try:
                    transcript = YouTubeTranscriptApi.get_transcript(video_id)
                    text = " ".join([t['text'] for t in transcript])
                    results.append(f"Video: {title}\nTranscript: {text[:2000]}...")
                except Exception as e:
                    results.append(f"Video: {title}\nNo transcript available")
            
            return "\n\n".join(results)
        except Exception as e:
            return f"YouTube search error: {str(e)}"

youtube_tool = Tool(
    name="YouTube Search",
    func=YouTubeSearchTool().search,
    description="Useful for finding video explanations and lecture content"
)

In [39]:
# 1. Wikipedia Search Tool
wikipedia_retriever = WikipediaRetriever(load_max_docs=3)
wiki_tool = Tool(
    name="Wikipedia Search",
    func=lambda query: [doc.page_content[:2000] for doc in wikipedia_retriever.get_relevant_documents(query)],
    description="Useful for factual information about people, places, and historical events"
)

In [40]:
# 2. Community Forum Search (Custom Implementation)
class ForumSearchTool:
    def __init__(self, forum_links):
        self.forum_apis = {
            link: self._create_forum_client(link) 
            for link in forum_links
        }
    
    def _create_forum_client(self, url):
        # Implement custom API client for your forum
        return lambda query: requests.get(f"{url}/search?q={query}").json()
        
    def search(self, query):
        results = []
        for name, client in self.forum_apis.items():
            results.extend(client(query)[:3])  # Get top 3 from each forum
        return results
forum_tool = Tool(
    name="Community Forum Search",
    func=ForumSearchTool(["forum.example1.com", "forum.example2.org"]).search,
    description="Useful for finding discussions and solutions from technical communities"
)


In [41]:
# 3. Social Media Search
social_media_tool = Tool(
    name="Social Media Search",
    func=lambda query: DuckDuckGoSearchRun().run(f"site:twitter.com {query} OR site:reddit.com {query}"),
    description="Useful for finding recent public opinions and trends"
)

In [42]:
# 4. Multi-Engine Web Search
search = GoogleSerperAPIWrapper()  # Requires SERPER_API_KEY
ddg_search = DuckDuckGoSearchRun()

multi_search_tool = Tool(
    name="Hybrid Web Search",
    func=lambda query: "\n\n".join([
        "Google Results:\n" + search.run(query),
        "DuckDuckGo Results:\n" + ddg_search.run(query)
    ]),
    description="Combines results from multiple search engines for comprehensive web coverage"
)


ValidationError: 1 validation error for GoogleSerperAPIWrapper
  Value error, Did not find serper_api_key, please add an environment variable `SERPER_API_KEY` which contains it, or pass `serper_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error

In [ ]:
tools = [
    wiki_tool, 
    forum_tool, 
    social_media_tool, 
    multi_search_tool, 
    youtube_tool
]

agent = initialize_agent(
    tools, 
    llm, 
    agent_type="zero-shot-react-description", 
    verbose=True
)

# User query
query = "Your search query here"

# Get RAG response
response = agent.run(query)

print("RAG Response:\n", response)

In [ ]:
query = "Your search query here"

# Get RAG response
response = agent.run(query)

print("RAG Response:\n", response)